In [11]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def evaluate(col, score_df, answer=False):
    if answer:
        res = "answer"
    else:
        res = "result"
    def f1(row):
        scores = scorer.score(row['gold_answer'], row[f'{col}_{res}'])
        return scores['rougeL'].fmeasure
    def rc(row):
        scores = scorer.score(row['gold_answer'], row[f'{col}_{res}'])
        return scores['rougeL'].recall
    def pre(row):
        scores = scorer.score(row['gold_answer'], row[f'{col}_{res}'])
        return scores['rougeL'].precision
    score_df[f'{col}_f1'] = score_df.apply(f1, axis=1)
    score_df[f'{col}_rc'] = score_df.apply(rc, axis=1)
    score_df[f'{col}_pre'] = score_df.apply(pre, axis=1)

Long memory score

In [ ]:
import pandas as pd

number = 100
answer_df = pd.read_json("MSC_eval_0117.json", lines=True)
# answer_df = pd.read_json("MSC_score_v1.json", lines=True)
score_df = answer_df[:number]
evaluate('long_mem', score_df)
evaluate('long_mem_recall', score_df)

In [13]:

long_mem_means = score_df[['long_mem_f1', 'long_mem_rc', 'long_mem_pre']].mean()
long_mem_recall_means = score_df[['long_mem_recall_f1', 'long_mem_recall_rc', 'long_mem_recall_pre']].mean()
print(long_mem_means)
print(long_mem_recall_means)

long_mem_f1     0.018658
long_mem_rc     0.732957
long_mem_pre    0.009545
dtype: float64
long_mem_recall_f1     0.045314
long_mem_recall_rc     0.790497
long_mem_recall_pre    0.023830
dtype: float64


In [ ]:
score_df.to_json("MSC_score.json")

Short memory score

In [8]:
import pandas as pd

number = 100
answer_df = pd.read_json("s_MSC_eval.json", lines=True)
# answer_df = pd.read_json("s_MSC_score_v1.json", lines=True)
score_df = answer_df[:number]
evaluate('short_mem', score_df)

In [13]:
short_mem_means = score_df[['short_mem_f1', 'short_mem_rc', 'short_mem_pre']].mean()
print(short_mem_means)

short_mem_f1     0.031041
short_mem_rc     0.810377
short_mem_pre    0.016103
dtype: float64


In [ ]:
score_df.to_json("s_MSC_score.json")